In [ ]:
# @title imports
from google.colab import drive
drive.mount('/content/drive')
import os
import pandas as pd
import numpy as np
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

windowList = np.array([2, 3, 8, 13, 18, 23])
simPath = "/content/drive/MyDrive/dataset/veremi/veremiWithT2/simulationscsv"
bsmPath = "/content/drive/MyDrive/test/dismiss/"
preprocPath = bsmPath+"/preprocessing"
timePath = preprocPath+"/time"

paths = [simPath, bsmPath, preprocPath, timePath]
for p in paths:
  if os.path.exists(p) == False:
    os.mkdir(p)
  # print(os.path.exists(p), p)

In [ ]:
# @title Preprocessing
np.seterr('raise')
# separar comunicações completas por veículos
os.chdir(simPath)
simulations = pd.Series([f for f in os.listdir() if os.path.isfile(f)]).sort_values().reset_index(drop=True)[:]
tlist = []
for simul in simulations.values:
  t = simul.split('.')
  tlist.append(int(t[0][-3:]))
totalsim = len(simulations)
os.chdir(preprocPath)
if os.listdir():
  clist = []
  concluded = pd.Series([f for f in os.listdir() if os.path.isfile(f)], dtype=pd.StringDtype()).sort_values().reset_index(drop=True)
  duplicated = concluded.loc[concluded.str.contains('\(1\)') == True]
  if len(duplicated) > 0:
    for file in duplicated:
      filePath = preprocPath + "/" + file
      os.remove(filePath)
      concluded.drop(concluded.loc[concluded.str.contains('\(1\)') == True].index, inplace=True)
  for concl in concluded.values:
    c = concl.split('.')
    clist.append(int(c[0][-3:]))
else:
  clist = []
remainlist = list(set(tlist) - set(clist))
while remainlist:
  os.chdir(preprocPath)
  concluded = pd.Series([f for f in os.listdir() if os.path.isfile(f)], dtype=pd.StringDtype()).sort_values().reset_index(drop=True)
  clist = []
  for concl in concluded.values:
    c = concl.split('.')
    clist.append(int(c[0][-3:]))
  remainlist = list(set(tlist) - set(clist))
  if remainlist:
    szremainlist = len(remainlist)
    item = np.random.randint(szremainlist)
    sim = remainlist[item]
    # start preprocess
    start_time = time.time()
    os.chdir(simPath)
    data = pd.read_csv(simulations[sim], index_col=0)
    a = pd.Series(data['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
    b = pd.Series(data['sender'].unique().astype(int)).sort_values().reset_index(drop=True)
    vehicles = pd.concat([a, b], ignore_index=True).unique()
    size = len(vehicles)
    newdata = []
    dataset = []

    for idr, r in enumerate(vehicles):
      # if idr <= 0:
        for ids, s in enumerate(data['sender'].loc[data['receiver'] == r].astype(int).unique()):
          # if ids <= 0:
            print('\r',
                  'Processing simulation '+str(remainlist[item]),
                  ', vehicle '+str(idr+1)+' of '+str(size),
                  " in {:.2f}".format(time.time() - start_time)+' sec |',
                  ' faltam '+str(szremainlist)+' simulações.',
                  sep='', end='', flush=True)
            interaction = data.loc[(data['receiver'] == r) & (data['sender'] == s)]
            atktype = interaction['attackerType'].unique()[0]
            # fix atk_16 records
            if atktype == 16:
              atk_16 = []
              for idind, ind in enumerate(interaction.index):
                pxSnd = interaction['pxSnd'][ind]
                pySnd = interaction['pySnd'][ind]
                sxSnd = interaction['sxSnd'][ind]
                sySnd = interaction['sySnd'][ind]
                if idind > 0:
                  if sxSndOld != 0 and sxSndOld != 0:
                    if pxSndOld == pxSnd and pySndOld == pySnd:
                      atk_16.append(ind)
                pxSndOld = pxSnd
                pySndOld = pySnd
                sxSndOld = sxSnd
                sySndOld = sxSnd
              # print(atk_16)
              interaction = interaction.loc[atk_16]
            l = len(interaction)
            if l >= windowList.min():
              # print(r, s, l, atktype)
              newdata = np.append(newdata, [sim, r, s])
              for idind, ind in enumerate(interaction[0:windowList.max()].index):

                # feature 1
                # RSSI
                feat1 = interaction['RSSI'][ind]

                # feature 2
                # distance between sender and receiver
                pxRcv = interaction['pxRcv'][ind]
                pyRcv = interaction['pyRcv'][ind]
                pzRcv = interaction['pzRcv'][ind]
                pxSnd = interaction['pxSnd'][ind]
                pySnd = interaction['pySnd'][ind]
                pzSnd = interaction['pzSnd'][ind]
                pRcv = np.array((pxRcv, pyRcv, pzRcv))
                pSnd = np.array((pxSnd, pySnd, pzSnd))
                feat2 = np.linalg.norm(pRcv-pSnd)

                # feature3
                # estimated AoA
                yDiff = abs(pyRcv - pySnd)
                xDiff = abs(pxRcv - pxSnd)
                feat3 = np.arctan(yDiff/xDiff)
                                                  
                # feature4 conformidade de deslocamento
                # displacement compliance = dc
                sxSnd = interaction['sxSnd'][ind]
                sySnd = interaction['sySnd'][ind]
                szSnd = interaction['szSnd'][ind]
                sSnd = np.array((sxSnd, sySnd, szSnd))
                sendTime = interaction['sendTime'][ind]
                if idind == 0:
                  feat4 = np.nan
                if idind >= 1:
                  pSndNow = pSnd
                  pEstimated = pSndOld + (sSndOld * (sendTime - sendTimeOld))
                  feat4 = np.linalg.norm(pSndNow-pEstimated)
                pSndOld = pSnd
                sSndOld = sSnd
                sendTimeOld = sendTime
                
                newdata = np.append(newdata, [feat1, feat2, feat3, feat4])

              # completar a janela de 23 bsm
              if l < windowList.max():
                for i in range(windowList.max()-l):
                  newdata = np.append(newdata, [np.nan])

              newdata = np.append(newdata, atktype)
              dataset.append(newdata)
              newdata = []

              
                  
    columns  = ['sim', 'receiver', 'sender']
    for i in range(windowList.max()):
      columns.extend(['rssi'+str(i), 'distance'+str(i), 'aoa'+str(i), 'dc'+str(i)])
    columns.append('attackerType')
    df = pd.DataFrame(dataset, columns=columns)
    # display(df)

    os.chdir(preprocPath)
    print('\rSaving sim'+f"{remainlist[item]:03}"+'.csv', sep='', end='', flush=True)
    if os.path.exists(preprocPath+'/sim'+f"{remainlist[item]:03}"+'.csv') == False:
      df.to_csv('sim'+f"{remainlist[item]:03}"+'.csv')
    
    # log the time of preprocessing
    os.chdir(timePath)
    simTime = time.time() - start_time
    dataTime = {"simulation": "sim"+f"{remainlist[item]:03}", "time": simTime}
    try:
      timeDf = pd.read_csv('timePreprocessing.csv', index_col=0)
    except FileNotFoundError:
      timeDf = pd.DataFrame({"simulation": [dataTime['simulation']], "time": [dataTime['time']]})
      timeDf.to_csv('timePreprocessing.csv')
    else:
      if (timeDf['simulation'] == dataTime['simulation']).any():
        timeDf.loc[timeDf['simulation'] == dataTime['simulation'], ['time']] = dataTime['time']
        timeDf.to_csv('timePreprocessing.csv')
      else:    
        timeDf = timeDf.append(dataTime, ignore_index=True)
        timeDf.to_csv('timePreprocessing.csv')

    # retira o item da remainlist
    remainlist.pop(item)
    del clist

os.chdir(timePath)
timeDf = pd.read_csv('timePreprocessing.csv', index_col=0)
totalTime = timeDf['time'].sum()
print('\rTotal time done in ', int(totalTime//3600), ':', "{:02d}".format(int(np.ceil(((totalTime/3600)%1)*60))), ' h', sep='')